## Prototype for first labeling interface
### Step 1
- Generate time series Test Data
    - Ensure columns are datetime,sensors,label
- Display the time series data using dash
    - Explore the new options
- Display the label column intuitively across the dash app somewhere
    - explore
- Result
    - Easy intuitive function to select region on dash graph and choose a corresponding label

### Step 2
- Explore Video & Sync Stuff


### Generate Some Synthetic Data

In [10]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# params
num_samples = 100
start_datetime = datetime(2023, 1, 1, 0, 0, 0)
labels = ['run', 'walk', 'sit']

# generating synthetic time series data
np.random.seed(0)  # reproduceability

# generate datetimes
datetimes = [start_datetime + timedelta(seconds=i) for i in range(num_samples)]

# generate sensor data with logic: run (higher values), walk (moderate values), sit (lower values)
sensor_data = []
for _ in range(num_samples):
    label = np.random.choice(labels)
    if label == 'run':
        x, y, z = np.random.uniform(0.6, 1.0, 3)
    elif label == 'walk':
        x, y, z = np.random.uniform(0.2, 0.5, 3)
    else:  # sit
        x, y, z = np.random.uniform(0.0, 0.2, 3)
    sensor_data.append([x, y, z, label])

# Creating the DataFrame
df = pd.DataFrame(sensor_data, columns=['x', 'y', 'z', 'label'])
df['datetime'] = datetimes
df = df[['datetime', 'x', 'y', 'z', 'label']]  # reorder cols
df = df.sort_values(by='datetime')

df


,datetime,x,y,z,label
0,2023-01-01 00:00:00,0.837138,0.937706,0.943178,run
1,2023-01-01 00:00:01,0.387069,0.315315,0.289260,walk
2,2023-01-01 00:00:02,0.985465,0.753377,0.916690,run
3,2023-01-01 00:00:03,0.113609,0.185119,0.014207,sit
4,2023-01-01 00:00:04,0.394452,0.310472,0.487147,walk
...,...,...,...,...,...
95,2023-01-01 00:01:35,0.283279,0.238658,0.317803,walk
96,2023-01-01 00:01:36,0.674852,0.961594,0.817522,run
97,2023-01-01 00:01:37,0.182109,0.016594,0.160076,sit
98,2023-01-01 00:01:38,0.018692,0.085261,0.094644,sit


In [11]:
import dash
from dash import html, dcc
import plotly.express as px
from pandas import DataFrame
import plotly.graph_objs as go

# Initialize Dash app
app = dash.Dash(__name__)

# Create a plotly graph object for each sensor
trace_x = go.Scatter(x=df['datetime'], y=df['x'], mode='lines', name='Sensor X')
trace_y = go.Scatter(x=df['datetime'], y=df['y'], mode='lines', name='Sensor Y')
trace_z = go.Scatter(x=df['datetime'], y=df['z'], mode='lines', name='Sensor Z')

# Combine traces in a single figure
fig = go.Figure(data=[trace_x, trace_y, trace_z])
fig.update_layout(title='Sensors Data Over Time', xaxis_title='Datetime', yaxis_title='Sensor Value')

# Update Dash layout to include this figure
app.layout = html.Div(children=[
    html.H1(children='Combined Sensor Data Visualization'),
    html.Div(children='''Time Series Data of Sensors x, y, z.'''),

    dcc.Graph(
        id='combined-sensor-graph',
        figure=fig
    )
])

# Run the app
if __name__ == '__main__':
    app.run_server(jupyter_mode="external")


Dash app running on http://127.0.0.1:8050/


In [14]:
import dash
from dash import html, dcc
import plotly.graph_objs as go
import pandas as pd

# Assume df is your DataFrame containing the data

# Prepare label colors
label_colors = {'run': 'red', 'walk': 'green', 'sit': 'blue'}

# Create a plotly graph object for each sensor
trace_x = go.Scatter(x=df['datetime'], y=df['x'], mode='lines', name='Sensor X')
trace_y = go.Scatter(x=df['datetime'], y=df['y'], mode='lines', name='Sensor Y')
trace_z = go.Scatter(x=df['datetime'], y=df['z'], mode='lines', name='Sensor Z')

# Initialize the figure with sensor data
fig_sensors = go.Figure(data=[trace_x, trace_y, trace_z])

# Add rectangles for labels
shapes = []
for label, group in df.groupby((df['label'] != df['label'].shift()).cumsum()):
    start_time = group['datetime'].iloc[0]
    end_time = group['datetime'].iloc[-1]
    label_color = label_colors[group['label'].iloc[0]]

    shapes.append({
        'type': 'rect',
        'x0': start_time,
        'y0': 0,
        'x1': end_time,
        'y1': 1,
        'xref': 'x',
        'yref': 'paper',
        'fillcolor': label_color,
        'opacity': 0.2,
        'line': {
            'width': 0,
        }
    })

# Add dummy traces for the legend
for label, color in label_colors.items():
    fig_sensors.add_trace(go.Scatter(
        x=[None],
        y=[None],
        mode='lines',
        name=label.capitalize(),
        line=dict(color=color)
    ))

# Update layout with shapes and titles
fig_sensors.update_layout(
    title='Sensors Data Over Time with Labels',
    xaxis_title='Datetime',
    yaxis_title='Sensor Value',
    shapes=shapes,
    legend_title='Activity Labels'
)

# Update Dash layout to include the figure
app.layout = html.Div(children=[
    html.H1(children='Sensor Data Visualization with Labels'),
    dcc.Graph(id='sensor-graph', figure=fig_sensors)
])

# Run the app
if __name__ == '__main__':
    app.run(jupyter_mode="external")


Dash app running on http://127.0.0.1:8050/


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   datetime  100 non-null    datetime64[ns]
 1   x         100 non-null    float64       
 2   y         100 non-null    float64       
 3   z         100 non-null    float64       
 4   label     100 non-null    object        
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 4.0+ KB


In [ ]:
#df
#labelListdf
#drive_link

